In [57]:
import cv2 as cv
import numpy as np
import os

In [16]:
img = cv.imread("test_data_stage_1_76perf/test_data_stage_1/DataTest64.png", )
converted = cv.convertScaleAbs(img, alpha=2.0, beta=0.0)
cv.imshow('contrast', converted)
cv.imshow('original', img)
cv.waitKey(0)
cv.destroyAllWindows()

In [54]:
def srgbtolin(x):
    if x <= 0.04045:
        return x / 12.92
    else:
        return ((x + 0.055) / 1.055) ** 2.4

luminance_res = []
for i in range(1, 101):
    img_rgb = cv.imread(f"DataTest/DataTest{i}.png")
    b, g, r = cv.split(img_rgb / 255)
    b_lin = np.vectorize(srgbtolin)(b)
    g_lin = np.vectorize(srgbtolin)(g)
    r_lin = np.vectorize(srgbtolin)(r)
    avg_luminance = np.average(b_lin * 0.0722 + g_lin * 0.7152 + r_lin * 0.2126)
    luminance_res.append((i, avg_luminance))
sorted_luminance = sorted(luminance_res, key=lambda x: x[1])


In [61]:
def calculate_luminance_avg(img_rgb):
    b, g, r = cv.split(img_rgb / 255)
    b_lin = np.vectorize(srgbtolin)(b)
    g_lin = np.vectorize(srgbtolin)(g)
    r_lin = np.vectorize(srgbtolin)(r)
    avg_luminance = np.average(b_lin * 0.0722 + g_lin * 0.7152 + r_lin * 0.2126)
    return avg_luminance

In [52]:
i = 1
for id, luminance in sorted_luminance:
    img = cv.imread(f"DataTrain/DataTrain{id}.png")
    cv.imwrite(f"luminance-train/{i}_DataTrain{id}.png", img)
    i += 1

#### Thresholding luminance

In [77]:
folder_path = "test_data_stage_1_76perf/test_data_stage_1"
dest_path = "test_data_contrast_76"
threshold = 0.1273920940530704 # ini dari hasil ngurutin trus diambil bates yg kira2 bagus wkwk
filenames = os.listdir(folder_path)
for filename in filenames:
    if filename == "test.txt":
        continue
    img = cv.imread(f"{folder_path}/{filename}")
    if calculate_luminance_avg(img) < threshold:
        converted = cv.convertScaleAbs(img, alpha=2.0, beta=0.0)
        # cv.imwrite(f"luminance/only-threshold-train/{filename[:-4]}_lum.png", converted)
        cv.imwrite(f"{dest_path}/{filename}", converted)
    else:
        cv.imwrite(f"{dest_path}/{filename}", img)
    #     cv.imwrite(f"luminance/only-threshold-test/{filename}", img)